## Imports

In [1]:
import torch
import shutil
import random
import glob

import pandas as pd
import numpy as np
import torchvision
from pathlib import Path
from PIL import Image

from torch import optim
from torch import nn

import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision.datasets import ImageNet
from torchvision import transforms
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

#import torchvision.models as models
#from torchvision.transforms import Compose
#from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

## Data sorting

In [9]:
class Resize(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation
        self.aspect_ratio = []

    def __call__(self, img):
        old_size = img.size  # old_size[0] is in (width, height) format

        ratio = float(self.size)/max(old_size)
        self.aspect_ratio.append(ratio)
        new_size = tuple([int(x * ratio) for x in old_size])
        
        img=img.resize(new_size, resample=self.interpolation)
        new_im = Image.new("RGB", (self.size, self.size))
        new_im.paste(img, ((self.size-new_size[0])//2,(self.size-new_size[1])//2))
        return new_im
#resize = Resize(224)
#img = resize(Image.open('train_all_test/00010.jpg'))
#img.save('train_all_test/00010.jpg') 


In [10]:
f2  = open("fname_string.txt", "r").read().split("\n")

resize = Resize(224)
for fname in f2:
    img = resize(Image.open('train_all/{}'.format(fname)))
    img.save('train_all/{}'.format(fname)) 
AR=resize.aspect_ratio
with open('aspectratio.txt', 'w') as f:
    for item in AR:
        f.write("%s\n" % item)

In [12]:
train_all_path = Path.cwd() / "train_all"

all_filenames = list(train_all_path.glob("*.jpg"))
all_filenames_length = len(all_filenames)
#print(all_filenames)

train_filenames = random.sample(all_filenames, int(0.8*all_filenames_length))
#print(train_filenames)
val_filenames=[]
for file in all_filenames:
    if file not in train_filenames:
        val_filenames.append(file)
        
        
for subdirectory in range(1,197):
    subdirectory = Path("train/{}".format(subdirectory))
    subdirectory.mkdir(parents=True, exist_ok=True)
for subdirectory in range(1,197):
    subdirectory = Path("val/{}".format(subdirectory))
    subdirectory.mkdir(parents=True, exist_ok=True)
    
f1  = open("class_string.txt", "r").read().split("\n")
     
for index in range(all_filenames_length):
    rand = np.random.rand()
    if rand < 0.8:
        shutil.copyfile(all_filenames[index], Path.cwd() / "train/{}/{}".format(f1[index],f2[index]))
    else:
        shutil.copyfile(all_filenames[index], Path.cwd() / "val/{}/{}".format(f1[index],f2[index]))
    

In [70]:
f3 = open("bbox.txt", "r").read().split("\n")
f1  = open("class_string.txt", "r").read().split("\n")
f4  = open("aspectratio.txt", "r").read().split("\n")
AR=[]
for line in f4:
    try:
        AR.append(float(line))
    except:
        pass
totbbox = []
for i,line in enumerate(f3):
    bbox = line.split("\t") # x1,y1,x2,y2
    bbox[0] = int(int(bbox[0])*AR[i])
    bbox[1] = int(int(bbox[1])*AR[i])
    bbox[2] = int(int(bbox[2])*AR[i])
    bbox[3] = int(int(bbox[3])*AR[i])
    totbbox.append(bbox)
    f1[i] = int(f1[i])
#totbbox = torch.tensor(totbbox,dtype=torch.float32)

fin_list = []
for i in range(len(f2)):
    fin_list.append([f2[i],totbbox[i][0],totbbox[i][1],totbbox[i][2],totbbox[i][3]])
print(fin_list[0][:])
df = pd.DataFrame(fin_list, columns=['image_id', 'x0','y0','x1','y1'])

['00001.jpg', 14, 43, 212, 140]


## Data Handler


In [2]:
def compare_transforms(transformations, index):
    """Visually compare transformations side by side.
    Takes a list of ImageFolder datasets with different compositions of transformations.
    It then display the `index`th image of the dataset for each transformed dataset in the list.
    
    Example usage:
        compare_transforms([dataset_with_transform_1, dataset_with_transform_2], 0)
    
    Args:
        transformations (list(ImageFolder)): list of ImageFolder instances with different transformations
        index (int): Index of the sample in the ImageFolder you wish to compare.
    """
    
    # Here we combine two neat functions from basic python to validate the input to the function:
    # - `all` takes an iterable (something we can loop over, like a list) of booleans
    #    and returns True if every element is True, otherwise it returns False.
    # - `isinstance` checks whether a variable is an instance of a particular type (class)
    if not all(isinstance(transf, ImageFolder) for transf in transformations):
        raise TypeError("All elements in the `transformations` list need to be of type ImageFolder")
        
    num_transformations = len(transformations)
    fig, axes = plt.subplots(1, num_transformations)
    
    # This is just a hack to make sure that `axes` is a list of the same length as `transformations`.
    # If we only have one element in the list, `plt.subplots` will not create a list of a single axis
    # but rather just an axis without a list.
    if num_transformations == 1:
        axes = [axes]
        
    for counter, (axis, transf) in enumerate(zip(axes, transformations)):
        axis.set_title("transf: {}".format(counter))
        image_tensor = transf[index][0]
        display_image(axis, image_tensor)

    plt.show()

def display_image(axis, image_tensor):
    """Display a tensor as image
    
    Example usage:
        _, axis = plt.subplots()
        some_random_index = 453
        image_tensor, _ = train_dataset[some_random_index]
        display_image(axis, image_tensor)
    
    Args:
        axis (pyplot axis)
        image_tensor (torch.Tensor): tensor with shape (num_channels=3, width, heigth)
    """
    
    # See hint above
    if not isinstance(image_tensor, torch.Tensor):
        raise TypeError("The `display_image` function expects a `torch.Tensor` " +
                        "use the `ToTensor` transformation to convert the images to tensors.")
        
    # The imshow commands expects a `numpy array` with shape (3, width, height)
    # We rearrange the dimensions with `permute` and then convert it to `numpy`
    image_data = image_tensor.permute(1, 2, 0).numpy()
    height, width, _ = image_data.shape
    axis.imshow(image_data)
    axis.set_xlim(0, width)
    # By convention when working with images, the origin is at the top left corner.
    # Therefore, we switch the order of the y limits.
    axis.set_ylim(height, 0)

In [48]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# For training
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
print(images)
print(boxes.shape)
print(labels.shape)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
print(targets)
output = model(images, targets)
print(output)

[tensor([[[0.4422, 0.5564, 0.1337,  ..., 0.3464, 0.0750, 0.2763],
         [0.3008, 0.9526, 0.8631,  ..., 0.6626, 0.9081, 0.0346],
         [0.0091, 0.1780, 0.3625,  ..., 0.2065, 0.6453, 0.0682],
         ...,
         [0.2500, 0.4008, 0.1139,  ..., 0.8823, 0.5413, 0.2733],
         [0.1238, 0.7298, 0.7411,  ..., 0.2676, 0.9307, 0.2228],
         [0.3523, 0.1417, 0.7861,  ..., 0.7813, 0.6794, 0.9571]],

        [[0.6697, 0.9908, 0.6568,  ..., 0.3863, 0.8997, 0.5018],
         [0.5269, 0.2296, 0.1138,  ..., 0.1567, 0.5222, 0.3227],
         [0.9658, 0.8961, 0.1739,  ..., 0.4766, 0.1538, 0.9061],
         ...,
         [0.4800, 0.8119, 0.6043,  ..., 0.0031, 0.9129, 0.0232],
         [0.0731, 0.1103, 0.6099,  ..., 0.8948, 0.5263, 0.5219],
         [0.6220, 0.8854, 0.8412,  ..., 0.8091, 0.6535, 0.8053]],

        [[0.7213, 0.2984, 0.3505,  ..., 0.3394, 0.6020, 0.5860],
         [0.7458, 0.9605, 0.6568,  ..., 0.5303, 0.7131, 0.5549],
         [0.5483, 0.7315, 0.2568,  ..., 0.9959, 0.1906, 0

In [62]:
train_path = Path.cwd() / "train"
val_path = Path.cwd() / "val"

transform = transforms.Compose([transforms.ToTensor()])
transform1 = transforms.Compose([transforms.ToTensor()])
train_folder = ImageFolder(train_path, transform=transform)
train_folder1 = ImageFolder(train_path, transform=transform1)
val_folder = ImageFolder(val_path, transform=transform)

train_loaded = DataLoader(train_folder, batch_size=32, shuffle = True)
val_loaded = DataLoader(val_folder, batch_size=32)
compare_transforms([train_folder, train_folder1], 0)


NameError: name 'compare_transforms' is not defined

## Setting up architecture


In [71]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
classes = 196
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

class CarDataset(Dataset):
    def __init__(self, df, image_dir, labels, transforms= None):
        super().__init__()
        self.df = df
        self.image_ids = self.df['image_id']
        self.image_dir = image_dir
        self.labels = labels
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image_id"]==image_id]
        img = Image.open(self.image_dir/image_id)
        img = transforms.ToTensor()(img)
        boxes = records[["x0","y0", "x1", "y1"]].values
        boxes = torch.tensor(boxes, dtype=torch.int64)
        labels = torch.tensor(self.labels[idx])
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        return img, target, image_id

                            

                                  
    def __len__(self):
        return self.image_ids.shape[0]

dataset = CarDataset(df, Path.cwd() / "train_all", f1)
datloaded = DataLoader(dataset, batch_size=32)        
print(datloaded)


## Training


In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);
num_epochs = 1
for epoch in range(num_epochs):
    for b_x, b_y, b_z in datloaded:
        b_x,b_y = b_x.to(device),{'boxes':b_y["boxes"].to(device),'labels': b_y["labels"].to(device)}
        print(b_x.shape)
        b_y['boxes'] = np.squeeze(b_y['boxes'])
        print(b_y)
        print(b_y)
        out_put=model(b_x,b_y)
    

torch.Size([32, 3, 224, 224])
{'boxes': tensor([[ 14,  43, 212, 140],
        [  8,  28, 216, 146],
        [ 29,  38, 210, 133],
        [ 66,  41, 158, 116],
        [ 21,  56, 206, 154],
        [100, 112, 200, 162],
        [ 30,  28, 189, 138],
        [ 25,  27, 207, 143],
        [  3,  22, 222, 134],
        [  7,  38, 218, 128],
        [ 17,  32, 210, 137],
        [  2,  27, 223, 128],
        [ 13,  16, 187, 137],
        [  8,  68, 217, 151],
        [ 11,  26, 206, 132],
        [  9,  17, 213, 138],
        [ 33,  44, 201, 129],
        [  3,   8, 212, 164],
        [ 36,  42, 203, 113],
        [  3,  62, 215, 133],
        [  5,  46, 207, 112],
        [ 22,  57, 201, 120],
        [  4,  12, 213, 104],
        [ 11,  27, 212, 135],
        [ 12,  32, 210, 127],
        [ 10,  62, 213, 135],
        [ 28,  38, 215, 119],
        [ 16,  41, 211, 138],
        [  3,  19, 218, 157],
        [ 20,  27, 218, 130],
        [ 12,  69,  97, 123],
        [  5,  13, 211, 156]],

IndexError: list index out of range